<a href="https://colab.research.google.com/github/t1seo/AIFFEL/blob/master/FUNDAMENTALS/10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10. 🦄 가랏, 몬스터볼! 전설의 포켓몬 찾아 삼만리

## 10-1. 들어가며
- 다양한 피쳐가 있는 데이터셋을 밑바닥부터 샅샅이 뜯어보고, 전설의 포켓몬을 분류하기 위한 피쳐에는 무엇이 있는지 생각해 본다.
- 모델 학습을 시작하기 전 모든 컬럼에 대해 그래프 시각화, 피벗 테이블 등을 활용하며 다양한 방법으로 충분한 EDA를 진행한다.
- 모델 학습에 넣기 위해서 전처리가 필요한 범주형/문자열 데이터에 대한 전처리를 원-핫 인코딩 등으로 적절하게 진행한다.
- 전체 데이터셋을 train/test 데이터셋으로 나누고, 적절한 분류 모델(Decision Tree)을 선택해 학습시키며 베이스라인과 비교해본다.

## 10-2. 안녕, 포켓몬과 인사해!
포켓몬의 이름, 속성, 또는 공격력이나 방어력 등과 같은 스탯 값만을 가지고 전설의 포켓몬인지 아닌지를 구별해낼 수 있을까요?

오늘은 이러한 분류 문제를 풀기 위해 데이터를 밑바닥부터 샅샅이 뜯어보는 연습을 할 것입니다.
이러한 과정을 탐색적 **데이터 분석(Exploratory Data Analysis, 이하 EDA)**이라고 합니다.
EDA는 더 좋은 데이터 분석과 더 좋은 머신러닝 모델을 만들기 위해 필수적인 과정입니다.

더욱 의미 있는, 그리고 더 나은 성능의 모델을 만들어내기 위해서는 무엇을 고민해야 하는지, 그 흐름을 한번 따라와 보시길 바랍니다!
이러한 흐름에 익숙해진다면 어떤 데이터셋을 만나더라도 충분히 빠르고 치밀하게 데이터셋을 다룰 수 있게 될 것입니다.

## 10-3. 포켓몬, 그 데이터는 어디서 구할까?
- [[캐글] Pokemon with stats](https://www.kaggle.com/abcsds/pokemon)

This data set includes 721 Pokemon, including their **number, name, first and second type, and basic stats: HP, Attack, Defense, Special Attack, Special Defense, and Speed**. It has been of great use when teaching statistics to kids. With certain types you can also give a geeky introduction to machine learning.


*총 11개의 피쳐가 있으며, 각각은 포켓몬의 ID, 이름, 첫 번째 속성, 두 번째 속성, 스탯의 총합, HP, 공격력, 방어력, 특수 공격력, 특수 방어력, 그리고 속도 이다.*


This are the raw attributes that are used for calculating how much damage an attack will do in the games. This dataset is about the pokemon games (NOT pokemon cards or Pokemon Go).

The data as described by Myles O'Neill is:

- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round


The data for this table has been acquired from several different sites, including:

- [pokemon.com](https://www.pokemon.com/us/pokedex/)
- [pokemondb](https://pokemondb.net/pokedex)
- [bulbapedia](https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number)

One question has been answered with this database: The type of a pokemon cannot be inferred only by it's Attack and Deffence. It would be worthy to find which two variables can define the type of a pokemon, if any. Two variables can be plotted in a 2D space, and used as an example for machine learning. This could mean the creation of a visual example any geeky Machine Learning class would love.

In [ ]:
from google.colab import files
file_upload = files.upload()

In [ ]:
pokemon_data = 'Pokemon.csv'

## 10-4. 포켓몬 데이터 불러오기

### 라이브러리 가져오기

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline # 브라우저 내부(inline)에서 바로 그래프 그림 보여지게 한다
%config InlineBackend.figure_format = 'retina' # %matplotlib 뒤에 넣어주면 그래프를 더 높은 해상도로 출력한다.

### 데이터셋을 pandas로 불러오기

In [ ]:
# 연습을 위해 원본을 따로 만들어둔다
original_data = pd.read_csv(pokemon_data)

In [ ]:
pokemon = original_data.copy() # original로부터 deep copy
print(pokemon.shape)

In [ ]:
pokemon.head()

- 데이터셋은 총 800행, 13열로 이루어져 있다.
- 포켓몬은 총 800마리이고, 각 포켓몬을 설명하는 특성(feature)는 13개이다.

타겟으로 두고 확인할 데이터는 `Legendary (전설의 포켓몬인지 아닌지의 여부)`이므로 `Legendary == True` 값을 가지는 레전드 포켓몬 데이터셋은 `legendary` 변수에, `Legendary == False` 값을 가지는 일반 포켓몬 데이터셋은 `ordinary` 변수에 저장해 둔다.



In [ ]:
# 전설의 포켓몬 데이터셋
legendary = pokemon[pokemon["Legendary"] == True].reset_index(drop=True)
print(legendary.shape)
legendary.head()

- 전설의 포켓은은 65마리가 있다.

---

- `Legendary`가 `True`인 데이터만 따로 떼어내서 저장했다.
- `reset_index`는  인덱스를 다시 처음부터 재배열 해주는데 `drop=True`를 옵션으로 지정해주면 기존 인덱스는 버리고 재배열 해준다. 이 옵션을 지정해주지 않으면 기존 인덱스는 컬럼에 들어가버린다.

In [ ]:
# 일반 포켓몬 데이터셋
ordinary = pokemon[pokemon["Legendary"] == False].reset_index(drop=True)
print(ordinary.shape)
ordinary.head()

## 10-5. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (1) 결측치와 전체 칼럼


### 빈 데이터 확인하기

In [ ]:
# 컬럼별 결측치 확인
pokemon.isnull().sum()

- `Type 2` 컬럼에만 총 386개의 결측치가 있다. 두 번째 속성이 없는 포켓몬이 있는 것 같다.

데이터셋을 다룰 때 빈 데이터를 다루는 것은 매우 조심스러운 일입니다. 데이터셋의 성격에 따라 빈 데이터를 어떻게 다루어야 할지에 대한 방법이 달라지기 때문이죠.

## 전체 컬럼 이해하기
데이터가 800개이고, 컬럼이 13개로 큰 데이터셋이 아니므로, 최대한 모든 데이터들을 하나하나 이해해 볼 수 있겠습니다.

In [ ]:
# 컬럼 개수 출력
print(len(pokemon.columns))

In [ ]:
# 컬럼 이름 출력
pokemon.columns

- **#** : 포켓몬 Id number. 같은 포켓몬이지만 성별이 다른 경우 등은 같은 #값을 가진다. `int`
- **Name** : 포켓몬 이름. 포켓몬 각각의 이름으로, 이름 데이터는 800개의 포켓몬이 모두 다르다. (unique) `str`
- **Type 1** : 첫 번째 속성. 속성을 하나만 가지는 경우 Type 1에 입력된다. `str`
- **Type 2** : 두 번째 속성. 속성을 하나만 가지는 포켓몬의 경우 Type 2는 NaN(결측값)을 가진다. `str`
- **Total** : 전체 6가지 스탯의 총합. `int`
- **HP** : 포켓몬의 체력. `int`
- **Attack** : 물리 공격력. (scratch, punch 등) `int`
- **Defense** : 물리 공격에 대한 방어력. `int`
- **Sp. Atk** : 특수 공격력. (fire blast, bubble beam 등) `int`
- **Sp. Def** : 특수 공격에 대한 방어력. `int`
- **Speed** : 포켓몬 매치에 대해 어떤 포켓몬이 먼저 공격할지를 결정. (더 높은 포켓몬이 먼저 공격한다) `int`
- **Generation** : 포켓몬의 세대. 현재 데이터에는 6세대까지 있다. `int`
- **Legendary** : 전설의 포켓몬 여부. !! **Target feature** !! `bool`

## 10-6. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (2) ID와 이름

### # : ID number


In [ ]:
# 총 몇 종류의 # 값이 있는지 확인
len(set(pokemon['#']))  # set 사용

- 전체 데이터는 총 800개 인데 `#`을 집합으로 만든 자료형은 그보다 작은 721개의 데이터를 가지고 있다.

In [ ]:
pokemon[pokemon['#'] == 6]

- 예를 들어 #6의 포켓몬은 Charizard, CharizardMega Charizard X, CharizardMega Charizard Y 세 개로 나뉩니다.
- 기본 포켓몬인 Charizard로부터 시작해서 진화한 Mega Charizard가 있고, X, Y는 성별을 나타내는 것으로 보입니다.



In [ ]:
# 유일한 이름의 개수 확인
len(set(pokemon['Name']))

- 모든 이름들은 unique 하다.

## 10-7. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (3) 포켓몬의 속성

### Type 1 & Type 2 : 포켓몬의 속성

In [ ]:
# 무작위로 두 마리의 포켓몬 살펴보기
pokemon.loc[[6, 10]]

- 6번 포켓몬인 Charizard는 Fire와 Flying 속성 두 가지를, 8번 포켓몬인 Wartortle은 Water 속성 단 한 가지만 가진다.
- 속성은 기본적으로 하나, 또는 최대 두개까지 가질 수 있는 것을 알 수 있다.

In [ ]:
#  각 속성의 종류는 총 몇 가지인지 알아본다
print("Type 1: ", len(list(set(pokemon["Type 1"]))))
print("Type 2: ", len(list(set(pokemon["Type 2"]))))

- Type 2의 속성이 하나 더 많다.

In [ ]:
# 각자의 집합을 차집합해준다
set(pokemon["Type 2"]) - set(pokemon["Type 1"])

- 둘의 차집합은 `NaN`으로 `Type 1`과 `Type 2`에는 모두 같은 세트의 데이터가 들어가 있는 것을 확인할 수 있다.

In [ ]:
# 포켓몬들의 모든 타입을 types 변수에 저장
types = list(set(pokemon["Type 1"]))
types.sort()
print(len(types))
print(types)

In [ ]:
# 데이터가 비어있는 NaN의 개수 확인
pokemon["Type 2"].isna().sum() # pandas isna() 함수 사용

- 총 386개의 포켓몬은 속성을 하나만 가지고 나머지는 두 개의 속성을 가진다.
---
- 참고: [판다스 isna() 함수](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html)


### Type 1 데이터 분포 plot
일반 포켓몬과 전설의 포켓몬 속성 분포가 각각 어떤지 확인하겠습니다.
우리의 데이터는 일반 포켓몬보다 전설의 포켓몬 수가 매우 적은 불균형 데이터이기 때문에, 전설의 포켓몬은 따로 시각화해 주는 것이 좋을 것 같군요.

다음과 같이 `plt`의 `subplot`을 활용해서 두 개의 그래프를 한 번에 그리면서, 그래프는 `sns(seaborn)`의 `countplot`을 활용하겠습니다.
`countplot`은 말 그대로 데이터의 개수를 표시하는 플롯입니다.

In [ ]:
# countplot 그리기
plt.figure(figsize=(20, 12)) # 그래프 크기

plt.subplot(211) # 서브 플롯 1
sns.countplot(data=ordinary, x="Type 1", order=types).set_xlabel('') # 위에서 만든 types 순으로 정렬
plt.title("[All Pokemons]")

plt.subplot(212) # 서브 플롯 2
sns.countplot(data=legendary, x="Type 1", order=types).set_xlabel('') # 위에서 만든 types 순으로 정렬
plt.title("[Legendary Pokemons]")

plt.show()

- 일반 포켓몬에는 물(Water) 속성, 일반(Normal) 속성, 벌레(Bug) 속성 등이 많다.
- 반면, 전설의 포켓몬에는 용(Dragon) 속성, 신령(Psychic) 속성이 많다.

In [ ]:
#  피벗 테이블(pivot table)로 각 속성에 Legendary 포켓몬들이 몇 퍼센트씩 있는지 확인해 본다

# Type1별로 Legendary의 비율을 보여주는 피벗 테이블
pd.pivot_table(pokemon, index="Type 1", values="Legendary").sort_values(by=["Legendary"], ascending=False)

- Flying 타입이 50%로 높은 비율을 갖는다.

---

- 참고: [pandas.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

### Type 2 데이터 분포 plot

Type 2에는 `NaN`(결측값)이 존재했었습니다. Countplot을 그릴 때는 결측값은 자동으로 제외됩니다.

In [ ]:
# countplot 그리기
plt.figure(figsize=(20, 12)) # 그래프 크기

plt.subplot(211) # 서브 플롯 1
sns.countplot(data=ordinary, x="Type 2", order=types).set_xlabel('') # 위에서 만든 types 순으로 정렬
plt.title("[All Pokemons]")

plt.subplot(212) # 서브 플롯 2
sns.countplot(data=legendary, x="Type 2", order=types).set_xlabel('') # 위에서 만든 types 순으로 정렬
plt.title("[Legendary Pokemons]")

plt.show()

- Flying 속성의 경우 두 경우 다 가장 많지만, 일반 포켓몬에는 Grass, Rock, Poison같은 속성이 많은 반면 전설의 포켓몬은 하나도 없습니다.
- 대신 여전히 Dragon, Psychic과 더불어 Fighting과 같은 속성이 많습니다.

In [ ]:
# Type2별로 Legendary 의 비율을 보여주는 피벗 테이블
pd.pivot_table(pokemon, index='Type 2', values="Legendary").sort_values(by=['Legendary'], ascending=False)

- Type2에서는 Fire 속성 포켓몬의 Legendary 비율이 25%로 가장 높습니다.

## 10-8. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (4) 모든 스탯의 총합

### Total : 모든 스탯의 총합

In [ ]:
# 모든 스탯의 종류를 stats 변수에 저장
stats = ["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]
stats

실제로 6개 스탯의 총합과 데이터에 제공된 Total값이 맞는지 확인해 볼까요? **데이터 분석에서 검증은 필수죠!**

> 데이터 검증은 필수!!

In [ ]:
# 첫 번째 포켓몬에 대해 검증
name = pokemon.loc[0, "Name"]
print(f"#0 pokemon: {name}") # 첫번째 포켓몬 이름
# print("#0 pokemon: {}\n".format(pokemon.loc[0, "Name"]))
print("total: ", int(pokemon.loc[0, "Total"])) # 첫번째 포켓몬의 total
print("stat: ", list(pokemon.loc[0, stats])) # 첫번째 포켓몬의 stat

print("sum of all stats: ", sum(list(pokemon.loc[0, stats])))

- 첫 번째 포켓몬에 대해서는 Total 값이 318로 일치한다.

In [ ]:
pokemon[stats] # stats는 위에서 만든 리스트
pokemon[stats].values # 값들을 2차원 리스트로
pokemon[stats].sum(axis=1) # axis 1(가로) 방향으로 stat 합

# pokemon['Total'].values

In [ ]:
# 전체 포켓몬에 대해 Total 값이 stats의 총합과 같은지 확인해본다
sum(pokemon['Total'].values == pokemon[stats].values.sum(axis=1)) # 가로 방향으로 더해야 하므로 axis=1

- `Total` 값과 모든 `stats`의 총합은 포켓몬 전체 데이터 수와 같은 800마리이다.

### Total값에 따른 분포 plot
`Total`값과 `Legendary`는 어떤 관계가 있을지 확인해본다.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8) # 크기

sns.scatterplot(data=pokemon, x="Type 1", y="Total", hue="Legendary")
plt.show()

- `Legendary` 여부에 따라 색깔(hue)를 달리 하도록 했다.
- 점의 색깔을 보면 `Type 1` 별로 `Total` 값을 확인했을 때, 전설의 포켓몬이 주로 `Total` 스탯 값이 높다는 것이 확인 된다.

## 10-9. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (5) 세부 스탯

### 세부스탯: HP, Attack, Defense, Sp. Atk, Sp. Def, Speed

In [ ]:
# 각각의 스탯값에 대한 분포
figure, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(nrows=3, ncols=2)
figure.set_size_inches(12, 15)

sns.scatterplot(data=pokemon, y="Total", x="HP", hue="Legendary", ax=ax1)
sns.scatterplot(data=pokemon, y="Total", x="Attack", hue="Legendary", ax=ax2)
sns.scatterplot(data=pokemon, y="Total", x="Defense", hue="Legendary", ax=ax3)
sns.scatterplot(data=pokemon, y="Total", x="Sp. Atk", hue="Legendary", ax=ax4)
sns.scatterplot(data=pokemon, y="Total", x="Sp. Def", hue="Legendary", ax=ax5)
sns.scatterplot(data=pokemon, y="Total", x="Speed", hue="Legendary", ax=ax6)
plt.show()

### 각 스탯에 대한 분석

**HP, Defense, Sp. Def** (좌측 그래프들)
- 전설의 포켓몬은 주로 높은 스탯을 갖지만, 이 세 가지에서는 일반 포켓몬이 전설의 포켓몬보다 특히 높은 몇몇 포켓몬이 있었다.
- 그러나 그 포켓몬들도 Total 값은 특별히 높지 않은 것으로 보아 특정 스탯만 특별히 높은, 즉 특정 속성에 특화된 포켓몬들로 보인다. (ex. 방어형, 공격형 등)

**Attack, Sp. Atk, Speed** (우측 그래프들)
- 이 세 가지 스탯은 Total과 거의 비례한다. 전설의 포켓몬이 각 스탯의 최대치를 차지하고 있다.



## 10-10. 전설의 포켓몬? 먼저 샅샅이 살펴보자! (6) 세대

Generation은 각 포켓몬의 "세대"로, 현재 데이터셋에는 1~6세대의 포켓몬이 존재합니다.
각 세대에 대한 포켓몬의 수를 확인해 봅시다.

In [ ]:
plt.figure(figsize=(10, 8))  

plt.subplot(211)
sns.countplot(data=ordinary, x="Generation").set_xlabel('')
plt.title("[All Pkemons]")
plt.subplot(212)
sns.countplot(data=legendary, x="Generation").set_xlabel('')
plt.title("[Legendary Pkemons]")
plt.show()

- 전설의 포켓몬은 1, 2세대에는 많지 않았고, 3세대부터 많아졌다가, 6세대에 다시 줄어든 것을 확인할 수 있다.





## 10-11. 전설의 포켓몬과 일반 포켓몬, 그 차이는? (1) Total값

### 전설의 포켓몬의 Total값

In [ ]:
# 전설의 포켓몬들의 속성 Total값을 확인
fig, ax = plt.subplots()
fig.set_size_inches(8, 4)

sns.scatterplot(data=legendary, y="Type 1", x="Total")
plt.show()

- 여기서 한 가지 특징이 보이는데 **전설의 포켓몬들의 `Total` 값들이 특정 점에 몰려있다** 것이다.

In [ ]:
# 전설의 포켓몬이 가지는 `Total` 값들의 집합 확인
sorted(list(set(legendary["Total"])))

- 실제로도 단 9가지 값밖에 존재하지 않는다.

In [ ]:
# 전설의 포켓몬이 가지는 각 Total 값에 대한 포켓몬 수 확인
fig, ax = plt.subplots()
fig.set_size_inches(8, 4)

sns.countplot(data=legendary, x="Total")
plt.show()

In [ ]:
round(65 / 9, 2)

- 총 65마리의 전설의 포켓몬이 9개의 `Total` 값만 가진다는 것은, **약 7.22마리 까지는 같은 `Total` 스탯 값을 가진다**는 의미이다.



In [ ]:
# ordinary 포켓몬 확인
print(sorted(list(set(ordinary["Total"]))))

In [ ]:
len(sorted(list(set(ordinary["Total"]))))

In [ ]:
round(735 / 195, 2)

- 일반 포켓몬은 총 195가지의 Total 속성값을 가지고, 전체 일반 포켓몬은 735마리이므로, **약 3.77마리만 같은 `Total` 스탯 값을 가진다.**

이로써 다음 두 가지를 알 수 있습니다.

- `Total`값의 다양성은 일반 포켓몬이 전설의 포켓몬보다 두 배 가까이 된다. 즉 전설의 포켓몬의 `Total`값은 다양하지 않다.

    => *한 포켓몬의 Total 속성값이 전설의 포켓몬의 값들 집합에 포함되는지의 여부는 전설의 포켓몬임을 결정하는 데에 영향을 미친다.*



- 또한, 전설의 포켓몬의 `Total` 값 중에는 일반 포켓몬이 가지지 못하는 `Total`값이 존재한다. ex) 680, 720, 770, 780

    => *Total값은 전설의 포켓몬인지 아닌지를 결정하는 데에 이러한 방식으로도 영향을 미칠 수 있다.*

**즉, Total값은 legendary인지 아닌지를 예측하는 데에 중요한 컬럼일 것이라는 결론을 내릴 수 있습니다.**

## 10-12. 전설의 포켓몬과 일반 포켓몬, 그 차이는? (2) 이름

### 특정 단어가 들어가 있는 이름
데이터를 보고 있으면 한 가지 더 특이한 점이 있습니다. 바로 이름!

전설의 포켓몬들의 이름을 보면, 특정 단어가 들어가 있는 이름, 또는 긴 이름을 가진 경우가 많음을 확인할 수 있습니다.

In [ ]:
# 비슷한 이름을 가진 포켓몬들의 모임
n1, n2, n3, n4, n5 = legendary[3:6], legendary[14:24], legendary[25:29], legendary[46:50], legendary[52:57]
names = pd.concat([n1, n2, n3, n4, n5]).reset_index(drop=True)
names

- 모든 포켓몬이 각각 유일한(unique) 이름을 가지지만, 전설의 포켓몬 사이에서는 비슷한 이름이 다수 존재한다.
- 특히 아래 이름들은 모두 세트로 이름이 지어져 있다는 것을 확인하 수 있다.
    - "MewTwo", "Latias", "Latios", "Kyogre", "Groudon", "Rayquaza", "Kyurem" 등의 이름에서부터 그 앞에 성이 붙는다.

In [ ]:
# 이름에 forme가 들어가는 여러 가지 전설의 포켓몬이 있는 경우도 존재한다
formes = names[13: 23]
formes

- 이름에 forme이 들어가면 이는 전설의 포켓몬일 확률이 아주 높다.

### 긴 이름
위와 비슷한 이유로, 전설의 포켓몬은 이름의 길이도 긴 경우가 많습니다.
데이터셋에 이름 길이 컬럼을 생성해서 비교해 보도록 하겠습니다.

`legendary`와 `ordinary` 각각에 모두 `"name_count"`라는 이름의 길이를 나타내는 컬럼을 만들어줍니다. 파이썬 람다(lambda) 기능을 사용해 행마다 이름의 길이를 구하고, 이를 `"name_count"` 칼럼에 넣어주었습니다.

In [ ]:
# legendary에 name_count 컬럼 추가
legendary["name_count"] = legendary["Name"].apply(lambda i: len(i))
legendary.head()

In [ ]:
ordinary["name_count"] = ordinary["Name"].apply(lambda i: len(i))
ordinary.head()

In [ ]:
# name_count 컬럼이 어떤 특징을 갖는지 그래프로 확인
plt.figure(figsize=(12, 10))

plt.subplot(211)
sns.countplot(data=legendary, x="name_count").set_xlabel('')
plt.title("Legendary")

plt.subplot(212)
sns.countplot(data=ordinary, x="name_count").set_xlabel('')
plt.title("Ordinary")

plt.show()

- 위에서 볼 수 있듯이, **전설의 포켓몬은 16 이상의 긴 이름을 가진 포켓몬이 많은 반면, 일반 포켓몬은 10 이상의 길이를 가지는 이름의 빈도가 아주 낮다.**

In [ ]:
# 전설의 포켓몬 이름이 10 이상일 확률
print(round(len(legendary[legendary["name_count"] > 9]) / len(legendary) * 100, 2), "%")

In [ ]:
# 일반 포켓몬의 이름이 10 이상일 확률
print(round(len(ordinary[ordinary["name_count"] > 9]) / len(ordinary) * 100, 2), "%")

**전설의 포켓몬의 이름이 10 이상일 확률은 41%** 를 넘는 반면에, **일반 포켓몬의 이름이 10 이상일 확률은 약 16%** 밖에 안됨을 확인할 수 있습니다!

이는 아주 큰 차이이므로 legendary인지 아닌지를 구분하는 데에 큰 의미가 있습니다.

위의 두 가지, 이름에 대한 분석은 중요한 시사점을 가집니다.

- **만약 "Latios"가 전설의 포켓몬이라면, "%%% Latios" 또한 전설의 포켓몬이다!**
- **적어도 전설의 포켓몬에서 높은 빈도를 보이는 이름들의 모임이 존재한다!**
- **전설의 포켓몬은 긴 이름을 가졌을 확률이 높다!**

## 10-13. 모델에 넣기위해! 데이터 전처리하기 (1) 이름의 길이가 10 이상인가?

데이터 분석을 통해 머신러닝을 수행하고 싶다면, **데이터를 모델에 입력할 수 있는 형태로 변환하는 것이 매우 중요합니다.**

머신러닝을 수행할 모델은 문자열 데이터를 처리할 수 없기 때문에 이를 적절한 숫자 데이터 또는 True, False를 나타내는 부울(bool) 데이터 등으로 전처리하는 과정이 필요합니다.
따라서 지금까지 수행한 EDA 결과에 따라 이름 컬럼을 모델이 연산할 수 있는 형태로 처리를 해 보도록 하겠습니다.

앞서 확인한 EDA 과정에서 이름은 전설의 포켓몬인지 아닌지를 결정하는 중요한 특징 중 하나였죠.
따라서 이름에 관해서는 두 가지를 중점적으로 처리하겠습니다.

**1. 이름의 길이** : name_count 컬럼을 생성 후 길이가 10을 넘는지 아닌지에 대한 categorical 컬럼을 생성

**2. 토큰 추출** : legendary 포켓몬에서 많이 등장하는 토큰을 추려내고 토큰 포함 여부를 원-핫 인코딩(One-Hot Encoding)으로 처리

### (1) 이름의 길이가 10 이상인가 아닌가

In [ ]:
# 이름의 길이를 확인할 수 있는 name_count 컬럼을 생성한다. 
# 이전에 생성한 방법과 같지만, 이번엔 전체 데이터가 있는 pokemon 데이터 프레임에 생성한다.
pokemon["name_count"] = pokemon["Name"].apply(lambda i: len(i))
pokemon.head()

In [ ]:
# 이름의 길이가 10 이상이면 True, 미만이면 False를 가지는 long_name 컬럼을 생성
pokemon["long_namea"] = pokemon["name_count"] >= 10
pokemon.head()

- 전설의 포켓몬을 분류하는 데에 이름의 길잇값 자체를 가진 name_count 컬럼이 더 유리할지, 혹은 long_name이 더 유리할지는 아직 모릅니다.
- 따라서 모델을 학습시킬 때에는 두 가지를 모두 활용해 볼 것입니다.

## 10-14. 모델에 넣기위해! 데이터 전처리하기 (2) 이름에 자주 쓰이는 토큰 추출

다음으로 할 일은 전설의 포켓몬 이름에 가장 많이 쓰이는 토큰을 알아보고 이에 대한 새로운 컬럼을 만드는 것입니다.

이름에 어떤 토큰이 있으면 전설의 포켓몬일 확률이 높을지를 찾아보는 것이죠.

토큰을 추출하기에 앞서, 포켓몬의 이름에 대해 먼저 알아보겠습니다.

포켓몬의 이름은 총 네 가지 타입으로 나뉩니다.

1. 한 단어면 `ex. Venusaur`
2. 두 단어이고, 앞 단어는 두 개의 대문자를 가지며 대문자를 기준으로 두 부분으로 나뉘는 경우 `ex. VenusaurMega Venusaur`
3. 이름은 두 단어이고, 맨 뒤에 X, Y로 성별을 표시하는 경우 `ex. CharizardMega Charizard X`
4. 알파벳이 아닌 문자를 포함하는 경우 `ex. Zygarde50% Forme`

### 이름에 알파벳이 아닌 문자가 들어간 경우 전처리하기

이 중 가장 먼저 '알파벳이 아닌 문자'를 포함하는 경우를 처리하도록 하겠습니다.

어떤 문자열이 알파벳으로만 이루어져 있는지를 확인하고 싶을 때는 `isalpha()` 함수를 사용하면 편리합니다.

우리는 알파벳이 아닌 문자를 포함하는 이름을 걸러내고 싶은데, 주의할 점은 이름에 띄어쓰기가 있는 경우에도 `isalpha() = False`로 처리된다는 점입니다.


따라서 알파벳 체크를 위해 띄어쓰기가 없는 컬럼을 따로 만들어준 후, 띄어쓰기를 빈칸으로 처리해서 확인하도록 하겠습니다.

In [ ]:
# 알파벳 체크를 위해 띄어쓰기가 없는 컬럼을 따로 만들어주기